![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)
# Implementing hybrid search with Redis

Hybrid search is all about combining lexical search with semantic vector search to improve result relevancy. This notebook will cover 3 different hybrid search strategies with Redis:

1. Linear combination of scores from lexical search (BM25) and vector search (Cosine Distance)
2. Reciprocal Rank Fusion (RRF)
3. Client-Side Reranking with a cross encoder model

The Redis Query Engine supports a unified interface for hybrid search with the [FT.HYBRID](https://redis.io/docs/latest/commands/ft.hybrid) command introduced in Redis Open Source 8.4.0, prior to which hybrid searches were only possible using [the aggregations API](https://redis.io/docs/latest/develop/ai/search-and-query/advanced-concepts/aggregations/). RedisVL added an interface for FT.HYBRID in 0.13.0 (via `HybridQuery`), and provided an interface for the aggregation approach for Redis prior to 8.4.0 (via `AggregateHybridQuery`). This notebook will demonstrate the usage of both approaches.

## Requirements
- Redis 8.4.0+
- redisvl>=0.13.0
- redispy>=7.1.0

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/vector-search/02_hybrid_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Install Packages

In [ ]:
from redis.commands.search.aggregation import Reducer
%pip install "git+https://github.com/redis/redis-vl-python.git@feat/RAAE-1236/hybrid-search" nltk pandas sentence-transformers

### Data/Index Preparation
 
In this section:

1. We prepare the data necessary for our hybrid search implementations by loading a collection of movies. Each movie object contains the following attributes:
    - `title`
    - `rating`
    - `description`
    - `genre`
 
2. We generate vector embeddings from the movie descriptions. This allows users to perform searches that not only rely on exact matches but also on semantic relevance, helping them find movies that align closely with their interests.

3. After preparing the data, we populate a search index with these movie records, enabling efficient querying based on both lexical and vector-based search techniques.

Running remotely or in collab? Run this cell to download the necessary dataset.

In [ ]:
# NBVAL_SKIP
!git clone https://github.com/redis-developer/redis-ai-resources.git temp_repo
!mv temp_repo/python-recipes/vector-search/resources .
!rm -rf temp_repo

### Install Redis

For this tutorial you will need a running instance of Redis if you don't already have one.

#### Local Redis
Use the shell script below to download, extract, and install [Redis](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/apt/) directly from the Redis package archive for a Linux environment.

In [20]:
# NBVAL_SKIP
%%sh
sudo apt-get install lsb-release curl gpg
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
sudo chmod 644 /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update
sudo apt-get install redis

redis-server --version
redis-server --daemonize yes --loadmodule /usr/lib/redis/modules/redisearch.so

#### Alternative Redis Access (Cloud, Docker, other)
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis -p 6379:6379 redis:latest`

In [1]:
from packaging.version import Version

from redis import __version__ as redis_version
from redisvl import __version__ as redisvl_version


if Version(redis_version) < Version("7.1.0"):
    raise RuntimeError("redis-py version must be >= 7.1.0")

if Version(redisvl_version) < Version("0.13.0"):
    raise RuntimeError("redisvl version must be >= 0.13.0")

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [2]:
import os
import warnings

warnings.filterwarnings('ignore')

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

### Create redis client, load data, generate embeddings

In [3]:
from redis import Redis
from redisvl.redis.connection import RedisConnectionFactory

client = Redis.from_url(REDIS_URL)
client.ping()

if Version(client.info()["redis_version"]) < Version("8.4.0"):
    raise RuntimeError("Redis version must be >= 8.4.0")

installed_modules = RedisConnectionFactory.get_modules(client)
if "search" not in installed_modules:
    raise RuntimeError("Redisearch module is not installed")

In [4]:
import json

with open("resources/movies.json", 'r') as file:
    movies = json.load(file)

In [5]:
from redisvl.utils.vectorize import HFTextVectorizer
from redisvl.extensions.cache.embeddings import EmbeddingsCache


# load model for embedding our movie descriptions
model = HFTextVectorizer(
    model='sentence-transformers/all-MiniLM-L6-v2',
    cache=EmbeddingsCache(
        name="embedcache",
        ttl=600,
        redis_client=client,
    )
)

# embed movie descriptions
movie_data = [
    {
        **movie,
        "description_vector": model.embed(movie["description"], as_buffer=True)
    } for movie in movies
]

In [6]:
movie_data[:1]

[{'id': 1,
  'title': 'Explosive Pursuit',
  'genre': 'action',
  'rating': 7,
  'description': 'A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.',
  'description_vector': b'\x8df|=``\n;\xeb\x91\xb7;A\xcb~\xbd\x00e\xce\xbb\xc6\x16J=L\xa7?=\xf8v\x95<e\xfa\x06\xbe\x14Y\xcf=*\x07p=?\xdb\r\xbd\x99\xf2H\xbdue\xc6<L\xdfa=b8\x16\xbc\xe7\xd3\x13<@\xaa\x1c=\x0f\xef\x89<\xad\xb0-<\x91\xb2\x9f\xbcZ\x0b\xc3\xbd\xa1NR=tl\xf7\xbcM>\x17\xbee\x15\x05\xb9Ou\xbf<\xc3\xe2b\xba\xd2\xa6\xa8\xbd\x80\xdc\xec\xbcMc%=\xcf\xe7r\xbb\x15OG=A(\x85=e@\xa2\xbc.Z\xd0\xbdR%K\xbd\xdb\xed\x94\xbcf\xddH=|&F<\xd9*\xec<\x8f\xd8\x8d\xbd\xb7Z\x98<\x10\xa3\xa3=Gg3\xbd$\xcd\xbd\xbd\xba$\xf7;\x04\xf5z=\xfb\xb4\x8c=\x8b\x0e\xc6\xbdhI\x90\xbd\x0f\x16\xbd;}\xe7\x0c\xbd)3\xc9\xbc\x8a\xf8\xbb\xbc\x94&u\xbb.\x8f\xca<\x03\x80J=\n\xaf*=\x8eOU\xbd\xcc\xf0\x95\xbc\x15\x02\x19=1\xf4K<\xc8\xc2\t=C\x83\xac=\x9a\xd7\xb8\xbd\xf9\xb5\x9c\xbdE\x85\x18=\x9fd&=73\xf8<\xf1\xf7\x88<\x0ev\xf2\xbb/=[\xb

### Define Redis index schema

Below, we build a schema that represents our movie objects.

In [7]:
from redisvl.schema import IndexSchema
from redisvl.index import SearchIndex


schema = IndexSchema.from_dict({
  "index": {
    "name": "movies",
    "prefix": "movie",
    "storage": "hash"
  },
  "fields": [
    { "name": "title", "type": "text" },
    { "name": "description", "type": "text" },
    { "name": "genre", "type": "tag", "attrs": {"sortable": True}},
    { "name": "rating", "type": "numeric", "attrs": {"sortable": True}},
    {
        "name": "description_vector",
        "type": "vector",
        "attrs": {
            "dims": 384,
            "distance_metric": "cosine",
            "algorithm": "hnsw",
            "datatype": "float32"
        }
    }
  ]
})


index = SearchIndex(schema, client, validate_on_load=True)
index.create(overwrite=True, drop=True)

### Populate index

Load movie objects into Redis

In [8]:
index.load(movie_data)

['movie:01KC4A0R6HWRX5VT8E3TAJDPSQ',
 'movie:01KC4A0R6J1WMC3T5C12TBRQKV',
 'movie:01KC4A0R6J1WMC3T5C12TBRQKW',
 'movie:01KC4A0R6J1WMC3T5C12TBRQKX',
 'movie:01KC4A0R6J1WMC3T5C12TBRQKY',
 'movie:01KC4A0R6J1WMC3T5C12TBRQKZ',
 'movie:01KC4A0R6J1WMC3T5C12TBRQM0',
 'movie:01KC4A0R6J1WMC3T5C12TBRQM1',
 'movie:01KC4A0R6KNBNXZDDGR1WQ5K9W',
 'movie:01KC4A0R6KNBNXZDDGR1WQ5K9X',
 'movie:01KC4A0R6KNBNXZDDGR1WQ5K9Y',
 'movie:01KC4A0R6KNBNXZDDGR1WQ5K9Z',
 'movie:01KC4A0R6KNBNXZDDGR1WQ5KA0',
 'movie:01KC4A0R6KNBNXZDDGR1WQ5KA1',
 'movie:01KC4A0R6KNBNXZDDGR1WQ5KA2',
 'movie:01KC4A0R6KNBNXZDDGR1WQ5KA3',
 'movie:01KC4A0R6KNBNXZDDGR1WQ5KA4',
 'movie:01KC4A0R6KNBNXZDDGR1WQ5KA5',
 'movie:01KC4A0R6KNBNXZDDGR1WQ5KA6',
 'movie:01KC4A0R6KNBNXZDDGR1WQ5KA7']

# Hybrid Search Approaches

Now that our search index is populated and ready, we will build out a few different hybrid search techniques in Redis.

To start, we will use the `HybridQuery` class that accepts a text string and vector to automatically combine text similarity and vector similarity scores.

At a minimum, the `HybridQuery` class requires the following arguments:

```python
query = HybridQuery(
    text="your query string here",
    text_field_name="<name of the text field in the index to do text search in>",
    vector=<bytes or numeric array, ex: [0.1, 0.2, 0.3]>,
    vector_field_name="<name of the vector field in the index to compute vector similarity>",
)
```

This defaults to using the reciprocal rank fusion (RRF) method to combine scores, and only outputs the final keys and combined scores. A more common minimal usage might be:

```python
query = HybridQuery(
    text="your query string here",
    text_field_name="<name of the text field in the index to do text search in>",
    vector=<bytes or numeric array, ex: [0.1, 0.2, 0.3]>,
    vector_field_name="<name of the vector field in the index to compute vector similarity>",
    combination_method="RRF",
    rrf_window=20,
    yield_text_score_as="text_score",
    yield_vsim_score_as="vector_similarity",
    yield_combined_score_as="hybrid_score",
    return_fields=["<list of fields to return>"],
)
```

## 1. Linear Combination

The goal of this technique is to calculate a weighted sum of the text similarity score for our provided text search and the vector similarity score for our provided vector.

The FT.HYBRID API introduced in Redis 8.4.0 supports a linear combination of text and vector scores (accessible as of RedisVL 0.13.0 in `HybridQuery`), and it is also possible with the aggregations API, as of `Redis 7.4.x` (search version `2.10.5` - accessible as of RedisVl 0.5.0 in `AggregateHybridQuery`).

In [9]:
# Sample user query (can be changed for comparisons)
user_query = "action adventure movie with great fighting scenes against a dangerous criminal, crime busting, superheroes, and magic"

In [37]:
import pandas as pd

from redisvl.query.hybrid import HybridQuery

vector = model.embed(user_query, as_buffer=True)

query = HybridQuery(
	text=user_query,
	text_field_name="description",
	vector=vector,
	vector_field_name="description_vector",
	combination_method="LINEAR",
	yield_text_score_as="text_score",
	yield_vsim_score_as="vector_similarity",
	yield_combined_score_as="hybrid_score",
	return_fields=["title"],
)

results = index.hybrid_search(query)
pd.DataFrame(results[:3])

,text_score,title,vector_similarity,hybrid_score
0,9.1624524482,The Incredibles,0.677012234926,3.22264429891
1,5.02411250758,Skyfall,0.601227402687,1.92809293416
2,4.13361061261,Explosive Pursuit,0.695675313473,1.72705590321


Alternatively, for Redis versions prior to 8.4.0, we can use the aggregations API:

In [38]:
from redisvl.query.aggregate import AggregateHybridQuery

agg_query = AggregateHybridQuery(
    text=user_query,
    text_field_name="description",
    vector=vector,
    vector_field_name="description_vector",
    return_fields=["title"],
)

print(f"Query being executed:\n{agg_query._build_query_string()}")

results = index.query(agg_query)
pd.DataFrame(results[:3])

Query being executed:
(~@description:(action | adventure | movie | great | fighting | scenes | dangerous | criminal | crime | busting | superheroes | magic))=>[KNN 10 @description_vector $vector AS vector_distance]


,vector_distance,title,vector_similarity,text_score,hybrid_score
0,0.645975530148,The Incredibles,0.677012234926,9.1624524482,3.22264429891
1,0.797545194626,Skyfall,0.601227402687,5.02411250758,1.92809293416
2,0.608649373055,Explosive Pursuit,0.695675313473,4.13361061261,1.72705590321


### Choosing your stopwords for better queries
You can see that the user query string has been tokenized and certain stopwords like 'and', 'for', 'with', 'but', have been removed, otherwise you would get matches on irrelevant words.
RedisVL uses [NLTK](https://www.nltk.org/index.html) english stopwords as the the default. You can change which default language stopwords to use with the `stopwords` argument.
You specify a language, like 'german', 'arabic', 'greek' and many others, provide your own list of stopwords, or set it to `None` to not remove any.

Note that both `HybridQuery` and `AggregateHybridQuery` process stopwords identically.

In [27]:
# translate our user query to French and use nltk french stopwords
french_query_text = "Film d'action et d'aventure avec de superbes scènes de combat, des enquêtes criminelles, des super-héros et de la magie"

french_film_query = HybridQuery(
    text=french_query_text,
    text_field_name="description",
    vector=model.embed(french_query_text, as_buffer=True),
    vector_field_name="description_vector",
    stopwords="french",
)

print(french_film_query.query._search_query.query_string())

# specify your own stopwords
custom_stopwords = set([
    "a", "is", "the", "an", "and", "are", "as", "at", "be", "but", "by", "for",
    "if", "in", "into", "it", "no", "not", "of", "on", "or", "such", "that", "their",
    "then", "there", "these", "they", "this", "to", "was", "will", "with"
])

stopwords_query = HybridQuery(
    text=user_query,
    text_field_name="description",
    vector=vector,
    vector_field_name="description_vector",
    stopwords=custom_stopwords,
)

print(stopwords_query.query._search_query.query_string())

# don't use any stopwords
no_stopwords_query = HybridQuery(
    text=user_query,
    text_field_name="description",
    vector=vector,
    vector_field_name="description_vector",
    stopwords=None,
)

print(no_stopwords_query.query._search_query.query_string())

(~@description:(film | d\'action | d\'aventure | superbes | scènes | combat | enquêtes | criminelles | super\-héros | magie))
(~@description:(action | adventure | movie | great | fighting | scenes | against | dangerous | criminal | crime | busting | superheroes | magic))
(~@description:(action | adventure | movie | with | great | fighting | scenes | against | a | dangerous | criminal | crime | busting | superheroes | and | magic))


### Choosing your text scoring function and weights
There are different ways to calculate the similarity between sets of text. Options for text scoring functions are TFIDF, TFIDF.DOCNORM, BM25STD, BM25STD.NORM, BM25STD.TANH, DISMAX, DOCSCORE, and HAMMING; the default is BM25STD and is easy to configure with the `text_scorer` parameter. Just like changing you embedding model can change your vector similarity scores, changing your text similarity measure can change your text scores.

>  For more information about supported scoring algorithms, see [the Redis documentation on scoring](https://redis.io/docs/latest/develop/ai/search-and-query/advanced-concepts/scoring/).

When combining text and vector scores using a linear combination (`combination_method="LINEAR"` in `HybridQuery` and the only option for `AggregateHybridQuery`), you can control the relative balance of these scores with tunable parameters.

The FT.HYBRID API calculates the combined score as:

```python
hybrid_score = {alpha} * text_score + {beta} * vector_similarity
```

Where `alpha` and `beta` can be provided to `HybridQuery` via the `linear_alpha` and `linear_beta` parameters. If neither one is specified, FT.HYBRID defaults to `alpha=0.3` and `beta=0.7`. If only `alpha` is specified, `beta` is set to `1 - alpha`, and vice-versa.

`AggregateHybridQuery` defines the combined score in reverse as:

```python
hybrid_score = {1-alpha} * text_score + {alpha} * vector_similarity
```

Where the `alpha` parameter is configurable on the `AggregateHybridQuery` class. If not specified, it defaults to `0.7`.

Try changing the `text_scorer` and `linear_alpha` parameters in the query below to see how results may change.

In [39]:
tfidf_query = HybridQuery(
    text=user_query,
    text_field_name="description",
    vector=vector,
    vector_field_name="description_vector",
    text_scorer="TFIDF", # can be one of [TFIDF, TFIDF.DOCNORM, BM25, DISMAX, DOCSCORE, BM25STD]
    stopwords=None,
	combination_method="LINEAR",
    linear_beta=0.25, # weight the vector score lower
    return_fields=["title", "description"],
	yield_text_score_as="text_score",
    yield_vsim_score_as="vector_similarity",
    yield_combined_score_as="hybrid_score",
)

results = index.hybrid_search(tfidf_query)
pd.DataFrame(results[:3])

,text_score,title,description,vector_similarity,hybrid_score
0,6,Explosive Pursuit,A daring cop chases a notorious criminal acros...,0.695675313473,4.67391882837
1,6,Despicable Me,When a criminal mastermind uses a trio of orph...,0.651065170765,4.66276629269
2,6,Skyfall,James Bond returns to track down a dangerous n...,0.601227402687,4.65030685067


## 2. Reciprocal Rank Fusion (RRF)

Instead of relying on document scores like cosine similarity and BM25/TFIDF, we can fetch items and focus on their rank. This rank can be utilized to create a new ranking metric known as [Reciprocal Rank Fusion (RRF)](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf). RRF is powerful because it can handle ranked lists of different length, scores of different scales, and other complexities.

The FT.HYBRID API introduced in Redis 8.4.0 supports using RRF to combine results from text and vector queries (accessible as of RedisVL 0.13.0 in `HybridQuery`). Unless otherwise specified, RRF is the default combination method.

The parameters available to customize the behaviour of RRF are `rrf_window` and `rrf_constant`. The `rrf_window` parameter controls the size of the window over which the RRF score is calculated, and the `rrf_constant` parameter controls the constant used in the RRF formula. Try changing these parameters to see how results may change.

In [40]:
query = HybridQuery(
	text=user_query,
	text_field_name="description",
	vector=vector,
	vector_field_name="description_vector",
	combination_method="RRF",
	rrf_window=20,
	rrf_constant=60,
	yield_text_score_as="text_score",
	yield_vsim_score_as="vector_similarity",
	yield_combined_score_as="hybrid_score",
	return_fields=["title", "description"],
)

results = index.hybrid_search(query)
pd.DataFrame(results[:3])

,text_score,title,description,vector_similarity,hybrid_score
0,9.1624524482,The Incredibles,"A family of undercover superheroes, while tryi...",0.677012234926,0.032522474881
1,4.13361061261,Explosive Pursuit,A daring cop chases a notorious criminal acros...,0.695675313473,0.032266458496
2,4.13361061261,The Dark Knight,"Batman faces off against the Joker, a criminal...",0.673311859369,0.031498015873


### Client-side RRF for older Redis versions

When using Redis versions prior to 8.4.0, you can still perform RRF by fetching the top-k results from both the text and vector queries, and then fusing them together on the client-side.

In [41]:
def fuse_rankings_rrf(*ranked_lists, weights=None, k=60):
    """
    Perform Weighted Reciprocal Rank Fusion on N number of ordered lists.
    """
    item_scores = {}
    
    if weights is None:
        weights = [1.0] * len(ranked_lists)
    else:
        assert len(weights) == len(ranked_lists), "Number of weights must match number of ranked lists"
        assert all(0 <= w <= 1 for w in weights), "Weights must be between 0 and 1"
    
    for ranked_list, weight in zip(ranked_lists, weights):
        for rank, item in enumerate(ranked_list, start=1):
            if item not in item_scores:
                item_scores[item] = 0
            item_scores[item] += weight * (1 / (rank + k))
    
    # Sort items by their weighted RRF scores in descending order
    return sorted(item_scores.items(), key=lambda x: x[1], reverse=True)

In [42]:
# Below is a simple example of RRF over a few lists of numbers
fuse_rankings_rrf([1, 2, 3], [2, 4, 6, 7, 8], [5, 6, 1, 2])

[(2, 0.04814747488101534),
 (1, 0.032266458495966696),
 (6, 0.03200204813108039),
 (5, 0.01639344262295082),
 (4, 0.016129032258064516),
 (3, 0.015873015873015872),
 (7, 0.015625),
 (8, 0.015384615384615385)]

We'll want some helper functions to  construct our individual text and vector queries

In [43]:
# Function to create a vector query using RedisVL helpers for ease of use
from redisvl.query import VectorQuery, TextQuery


def make_vector_query(user_query: str, num_results: int, filters = None) -> VectorQuery:
    """Generate a Redis vector query given user query string."""
    vector = model.embed(user_query, as_buffer=True)
    query = VectorQuery(
        vector=vector,
        vector_field_name="description_vector",
        num_results=num_results,
        return_fields=["title", "description"]
    )
    if filters:
        query.set_filter(filters)
    return query


def make_ft_query(text_field: str, user_query: str, num_results: int) -> TextQuery:
    """Generate a Redis full-text query given a user query string."""
    return TextQuery(
        text=user_query,
        text_field_name=text_field,
        text_scorer="BM25",
        num_results=num_results,
        return_fields=["title", "description"],
    )

In [44]:
from typing import List, Dict, Any


def weighted_rrf(
    user_query: str,
    alpha: float = 0.5,
    num_results: int = 4,
    k: int = 60,
) -> List[Dict[str, Any]]:
    """Implemented client-side RRF after querying from Redis."""
    # Create the vector query
    vector_query = make_vector_query(user_query, num_results=len(movie_data))

    # Create the full-text query
    full_text_query = make_ft_query("description", user_query, num_results=len(movie_data))

    # Run queries individually
    vector_query_results = index.query(vector_query)
    full_text_query_results = index.query(full_text_query)

    # Extract titles from results
    vector_titles = [movie["title"] for movie in vector_query_results]
    full_text_titles = [movie["title"] for movie in full_text_query_results]

    # Perform weighted RRF
    return fuse_rankings_rrf(vector_titles, full_text_titles, weights=[alpha, 1-alpha], k=k)[:num_results]

In [45]:
# Test it out!
weighted_rrf(user_query, num_results=6)

[('Explosive Pursuit', 0.01639344262295082),
 ('The Dark Knight', 0.015873015873015872),
 ('Despicable Me', 0.015625),
 ('The Incredibles', 0.015417457305502846),
 ('Skyfall', 0.0152073732718894),
 ('Finding Nemo', 0.014242424242424244)]

But say we want to give more weight to the vector search rankings in this case to boost semantic similarities contribution to the final rank:

In [46]:
weighted_rrf(user_query, alpha=0.7, num_results=6)

[('Explosive Pursuit', 0.01639344262295082),
 ('The Dark Knight', 0.015873015873015872),
 ('The Incredibles', 0.015702087286527514),
 ('Despicable Me', 0.015625),
 ('Skyfall', 0.014838709677419354),
 ('Finding Nemo', 0.01387878787878788)]

## 3. Client-side reranking

An alternative approach to RRF is to simply use an external reranker to order the final recommendations. RedisVL has built-in integrations to a few popular reranking modules.

In [47]:
from redisvl.utils.rerank import HFCrossEncoderReranker

# Load the ms marco MiniLM cross encoder model from huggingface
reranker = HFCrossEncoderReranker("cross-encoder/ms-marco-MiniLM-L-6-v2")


def rerank(
    user_query: str,
    num_results: int = 4,
) -> List[Dict[str, Any]]:
    """Rerank the candidates based on the user query with an external model/module."""
    # Create the vector query
    vector_query = make_vector_query(user_query, num_results=num_results)

    # Create the full-text query
    full_text_query = make_ft_query("description", user_query, num_results=num_results)

    # Run queries individually
    vector_query_results = index.query(vector_query)
    full_text_query_results = index.query(full_text_query)

    # Assemble list of potential movie candidates with their IDs
    movie_map = {}
    for movie in vector_query_results + full_text_query_results:
        candidate = f"Title: {movie['title']}. Description: {movie['description']}"
        if candidate not in movie_map:
            movie_map[candidate] = movie

    # Rerank candidates
    reranked_movies, scores = reranker.rank(
        query=user_query,
        docs=list(movie_map.keys()),
        limit=num_results,
        return_score=True
    )

    # Fetch full movie objects for the reranked results
    return [
        (movie_map[movie['content']]["title"], score)
        for movie, score in zip(reranked_movies, scores)
    ]


In [48]:
# Test it out!
rerank(user_query, num_results=6)

[('The Incredibles', -4.163680076599121),
 ('Explosive Pursuit', 0.8551051616668701),
 ('The Dark Knight', -4.403158664703369),
 ('Skyfall', -7.830076217651367),
 ('Mad Max: Fury Road', -7.7119951248168945),
 ('Despicable Me', -8.742402076721191)]

This technique is certainly much slower than simple RRF as it's running an additional cross-encoder model to rerank the results. This can be fairly computationally expensive, but tunable with enough clarity on the use case and focus (how many items to retrieve? how many items to rerank? model accleration via GPU?)

## Post-processing configuration with FT.HYBRID

The FT.HYBRID API also allows for post-processing of the results (e.g. aggregations and aliasing).

In [110]:
from redis.commands.search import reducers

query = HybridQuery(
	text=user_query,
	text_field_name="description",
	vector=vector,
	vector_field_name="description_vector",
	combination_method="RRF",
	rrf_window=20,
	yield_text_score_as="text_score",
	yield_vsim_score_as="vector_similarity",
	yield_combined_score_as="hybrid_score",
	return_fields=["title", "genre", "description", "rating"],
	num_results=20,
)

query.postprocessing_config.group_by(
	"@genre",
	reducers.max("@hybrid_score").alias("max_hybrid_score"),
	reducers.avg("@hybrid_score").alias("avg_hybrid_score"),
	reducers.count().alias("count"),
	reducers.max("@rating").alias("max_rating"),
	reducers.min("@rating").alias("min_rating"),
).apply(
	rating_range="@max_rating - @min_rating",
)

results = index.hybrid_search(query)
pd.DataFrame(results)

,genre,max_hybrid_score,avg_hybrid_score,count,max_rating,min_rating,rating_range
0,comedy,0.0292869269949,0.0166705489146,10,8,6,2
1,action,0.032522474881,0.0271869721534,10,9,6,3


## Comparing Approaches

While each approach has strengths and weaknesses, it's important to understand that each might work better in some use cases than others. Below we will run through a sample of user queries and generate matches for each using different hybrid search techniques.

In [49]:
movie_user_queries = [
    "I'm in the mood for a high-rated action movie with a complex plot",
    "What's a funny animated film about unlikely friendships?",
    "Any movies featuring superheroes or extraordinary abilities", 
    "I want to watch a thrilling movie with spies or secret agents",
    "Are there any comedies set in unusual locations or environments?",
    "Find me an action-packed movie with car chases or explosions",
    "What's a good family-friendly movie with talking animals?",
    "I'm looking for a film that combines action and mind-bending concepts",
    "Suggest a movie with a strong female lead character",
    "What are some movies that involve heists or elaborate plans?",
    "I need a feel-good movie about personal growth or transformation",
    "Are there any films that blend comedy with action elements?", 
    "Show me movies set in dystopian or post-apocalyptic worlds",
    "I'm interested in a movie with themes of revenge or justice",
    "What are some visually stunning movies with impressive special effects?"
]

In [52]:
from typing import Tuple


def hybrid_query(text, num_results: int, **kwargs) -> List[Tuple[str, float]]:

    query = HybridQuery(
		text,
		text_field_name="description",
		vector=model.embed(text, as_buffer=True),
		vector_field_name="description_vector",
		stopwords="english",
		num_results=num_results,
		return_fields=["title"],
		yield_combined_score_as="hybrid_score",
		**kwargs,
    )

    results = index.hybrid_search(query)

    return [
        (
            movie["title"],
            movie["hybrid_score"]
        )
        for movie in results
    ]

In [58]:
import pandas as pd


rankings = pd.DataFrame()
rankings["query"] = movie_user_queries

# First, add new columns to the DataFrame
rankings["hf-cross-encoder"] = ""
rankings["rrf"] = ""
rankings["linear"] = ""

In [59]:
# Now iterate through the queries and add results
for i, user_query in enumerate(movie_user_queries):
    rankings.at[i, "hf-cross-encoder"] = rerank(user_query, num_results=4)
    rankings.at[i, "rrf"] = hybrid_query(user_query, num_results=4, combination_method="RRF", rrf_window=20)
    rankings.at[i, "linear"] = hybrid_query(user_query, num_results=4, combination_method="LINEAR", linear_alpha=0.3)

In [60]:
rankings.head()

,query,hf-cross-encoder,rrf,linear
0,I'm in the mood for a high-rated action movie ...,"[(Mad Max: Fury Road, -11.244140625), (Toy Sto...","[(The Incredibles, 0.032266458496), (Toy Story...","[(The Incredibles, 1.02685220568), (Toy Story,..."
1,What's a funny animated film about unlikely fr...,"[(Despicable Me, -10.44190788269043), (The Inc...","[(Monsters, Inc., 0.0312805474096), (Madagasca...","[(Madagascar, 1.23686656547), (Monsters, Inc.,..."
2,Any movies featuring superheroes or extraordin...,"[(The Incredibles, -3.6648061275482178), (The ...","[(The Incredibles, 0.0327868852459), (Mad Max:...","[(The Incredibles, 1.45202635245), (The Avenge..."
3,I want to watch a thrilling movie with spies o...,"[(Inception, -10.843632698059082), (The Incred...","[(Skyfall, 0.032266458496), (Explosive Pursuit...","[(Inception, 1.31241818816), (Skyfall, 0.44384..."
4,Are there any comedies set in unusual location...,"[(The Incredibles, -11.45376968383789), (Findi...","[(Finding Nemo, 0.0315449577745), (Explosive P...","[(Finding Nemo, 1.23817388011), (Madagascar, 0..."


In [61]:
rankings.loc[12].T

query               Show me movies set in dystopian or post-apocal...
hf-cross-encoder    [(Mad Max: Fury Road, -3.490626811981201), (De...
rrf                 [(The Incredibles, 0.032522474881), (Mad Max: ...
linear              [(The Incredibles, 1.3571597644), (Finding Nem...
Name: 12, dtype: object

# Wrap up
That's a wrap! Hopefully from this you were able to learn:
- How to implement simple vector search queries in Redis
- How to implement vector search queries with full-text filters
- How to implement hybrid search queries using the Redis hybrid and aggregation APIs
- How to perform client-side fusion and reranking techniques